# Raíces de funciones uni-dimensionales

En este notebook, investigaremos algunos métodos para encontrar **raíces** de funciones uni-dimensionales, usando métodos iterativos. Esto es un problema que ocurre por todos lados en la física, por ejemplo para el pozo cuadrado en mecánica cuántica, o para encontrar el valor máximo de una función.

Recuerda que $x^*$ es una **raíz** (también llamado "cero") de la función $f$ si $f(x^*) = 0$. Como sabemos, en general las raíces de una función no se pueden encontrar de manera analítica, excepto para funciones $f$ que sean polinomios de grado $\le 4$.

Por lo tanto, para encontrar raíces tendremos que utilizar algoritmos iterativos.
Recuerda que un algoritmo iterativo tiene la forma general

$$x_{n+1} := f(x_n),$$

y consiste en comenzar en un punto inicial $x_0$ y generar una secuencia $x_1 = f(x_0)$; $x_2 = f(x_1)$, etc.
Si diseñamos correctamente el algoritmo, la esperanza es que la secuencia $(x_n)_{n=1}^\infty$ converja a alguna raíz  $x^*$ con $f(x_\infty) = 0$, es decir que $f(x_n) \to 0$ cuando $n \to \infty$.

Dado que no podemos llevar a cabo la iteración un número infinito de veces, se corta la iteración después de un cierto número de pasos, para dar una solución *aproximada*, que se encuentra dentro de cierta *tolerancia* del resultado teórico exacto $x^*$. Por lo tanto, cualquier algoritmo iterativo requiere una condición de terminación.

# Tasa de convergencia de un algoritmo iterativo

Un tema crucial en torno a los métodos iterativos es saber no sólo si converge, sino también a *qué velocidad* converge, o con qué tasa converge - es decir, "qué tan rápido" va convergiendo a la solución.

**[1]** Considera la iteración $x_{n+1} = f(x_n)$. Supón que para algún valor de $n$, ya se acercó a un punto fijo $x_{\infty}$. Define la distancia $\delta_n := x_n - x_\infty$, que suponemos es chica.

(i) Sustituye esto en la fórmula de la iteración para encontrar una expresión aproximada que relaciona $\delta_{n+1}$ con la distancia al paso anterior, $\delta_n$, es decir, cómo se va variando el error. [Pista: ¿De cuál teorema famoso necesitas echar mano?]

(ii) Por lo tanto, encuentra una expresión aproximada explícita para $\delta_n$ en términos de $n$.

(iii) Verifica numéricamente el comportamiento de los $\delta_n$ con una iteración específica, por ejemplo con $f=\cos$.

(iv) ¿Cómo puedes graficar *de forma útil* el comportamiento de $\delta_n$ en función de $n$? Hazlo.

(v) Estima cuántas iteraciones se necesitan para llegar a que el error sea del orden de $2^{-p}$

(vi) Verifica esto numéricamente para $f=\cos$ usando `BigFloat`s con precisión $p=1000$ bits y verificando cuánto tiempo se requiere para que llegue a un punto fijo. [Recuerda que la precisión de los `BigFloat` se cambia a $p$ bits con `setprecision(p)`.]

In [1]:
function x_infinito(f, x0)
    
    x_actual = x0
    x_inf = 0
    A = []
    
    for i in 1:100
        
        x_nuevo = f(x_actual)
        
        if abs(x_actual - x_nuevo) < 0.000001
            x_inf = x_nuevo
            break
        end
        
        x_actual = x_nuevo
        push!(A,x_nuevo)
        
    end
    return x_inf, A
end

x_infinito (generic function with 1 method)

In [12]:
function convergencia(f, x0, d) 
    contador = 0
    while abs(x0 - f(x0)) > d
        x0 = f(x0)
        contador += 1
    end
    println("Número de iteraciones: $contador")
    return x0
end

convergencia (generic function with 1 method)

In [20]:
convergencia(cos, 1, 0.0000000000000001)

Número de iteraciones: 90


0.7390851332151607

In [4]:
function diferencia(f, x0)
    
    (x_inf,A) = x_infinito(f, x0)
    x_actual = x0
    B = []
    
    for i in 1:length(A)
        x_nuevo = f(x_actual)
            push!(B,x_nuevo)
        x_actual = x_nuevo
        
    end
    return abs.(B - x_inf)
end

diferencia (generic function with 1 method)

In [5]:
diferencia(cos, 1)

33-element Array{Float64,1}:
 0.198783  
 0.118468  
 0.0847957 
 0.0543948 
 0.0377168 
 0.0248742 
 0.0169831 
 0.0113322 
 0.00768148
 0.00515183
 0.00348079
 0.00233956
 0.00157864
 ⋮         
 4.46502e-5
 3.07356e-5
 2.00456e-5
 1.41611e-5
 8.88102e-6
 6.64037e-6
 3.81504e-6
 3.22784e-6
 1.51633e-6
 1.6794e-6 
 4.73286e-7
 9.76787e-7

# Raíz cuadrada: El algoritmo Babilónico

Un ejemplo de un algoritmo iterativo sorprendentemente eficaz y bonito es el *algoritmo Babilónico* (o de Herón) para calcular la raiz cuadrada $\sqrt{y}$ de un número real $y$ [cosa que probablemente nunca antes sabías hacer (más que tecleando una cierta tecla en una calculadora, ¡lo cual *no* cuenta!)].

**[2]** ¿De cuál ecuación es raíz el número $\sqrt{y}$? ¿Cuál otra solución de esta ecuación hay?

- $\sqrt{y}$ es solución de la ecuación $ay^2 + by + c = 0$ y la otra solución a la misma ecuación es $-\sqrt{y}$

Para un algoritmo, siempre necesitamos una *idea*, que toma una adivinanza $x_n$ y produce una (probablemente) mejor, $x_{n+1}$. La idea del algoritmo Babilónico es la siguiente.

**[3]** (i) Dada una adivinanza $x_n$, es posible (pero improbable) que $x_n$ ya sea igual a $\sqrt{y}$. ¿Cómo lo puedes verificar, sin utilizar (por supuesto) alguna función en Julia que calcule la raíz cuadrada? Escribe el código correspondiente.

(ii) Si $x_n$ no es raíz, demuestra (a mano) que $\frac{y}{x_n}$ se encuentra *del lado opuesto de $\sqrt{y}$ que $x_n$* sobre la recta real.

(iii) Así, tenemos dos valores que se encuentran por dos lados diferentes de $\sqrt{y}$. ¿Cuál sería una mejor adivinanza para $x_{n+1}$? 

- Intentaremos "adivinar" cuál es la solución de $\sqrt{49}$

In [6]:
y = 7
if y*y == 49
    println("$y es solución")
else 
    println("$y no es solución")
end

7 es solución


In [7]:
y = 5
if y*y == 49
    println("$y es solución")
else 
    println("$y no es solución")
end

5 no es solución


**[4]** (i) Utiliza esta idea para escribir una función que calcule $\sqrt{y}$ para una $y$ dada. ¿Cuál condición de terminación de la iteración te parece razonable? Impleméntalo.

(ii) Dibuja el comportamiento de $\delta_n$ en función de $n$. ¿Qué observas? ¿Qué implica esto sobre la eficiencia del algoritmo comparado con una iteración de punto fijo?

# Raíces de funciones: Bisección

Un primer método para encontrar una raíz de una función general $f$ es el **método de bisección**.
Dada una función continua $f$, una condición suficiente (pero no necesaria) para que *exista* una raiz en un intervalo dado $[a, b]$ es que $F$ cambie de signo en el intervalo, es decir, que $f(a)$ y $f(b)$ tengan signos opuestos. Si ocurre esto, entonces el teorema del valor intermedio nos dice que se sigue que $f$ sí tiene al menos una raíz en $[a, b]$.

**[5]** La idea del método de bisección es el siguiente. Lo implementaremos en una función `biseccion` que tome como argumento la función $f$ y el intervalo $[a, b]$.

(i) Verifica que $f(a)$ y $f(b)$ tengan signos opuestos. Si no, arroja un error.

(ii) Define $c$ como algún punto del intervalo. [Pista: ¿cuál sería uno sencillo?]

(iii) Esto divide el intervalo original en dos partes. Es posible (aunque improbable) que $c$ ya sea la raíz, en cuyo caso ya se termina la función y se regresa la raíz que hemos encontrado. ¿Cómo se checa si ya es la raíz? 

Si no, ¿cómo podemos saber en cuál de los dos sub-intervalos cae la raíz? Impleméntalo. 

[Pista: Probablmente querrás definir una función `signo` que toma un argumento `x` y regresa $0$ si `x` es igual a `0`, regresa `1` si `x` es mayor que cero, y regresa `-1` si `x` es menor que cero.]

(iv) Repite estos pasos hasta que encuentres la raíz con cierta tolerancia.

**[6]** Aplica tu función para encontrar *las dos* raíces cuadradas de $2$. Para hacerlo, tendrás que escoger (a mano) intervalos iniciales que cumplan con la condición de cambio de signo. 

**[7]** Haz una versión nueva de tu función que regresa un arreglo de todos los iterados `x`. Utiliza un método gráfico para estimar la tasa de convergencia del método hacia la raíz. 